In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm
import matplotlib.dates as mdates
from scipy.signal import find_peaks
import openmeteo_requests
import requests_cache
from datetime import date, timedelta


In [5]:
import sys
sys.path.append("../..")
from utils.helper_functions import *

In [6]:
# Einlesen von alle_fahrradzaehler_daten.csv
data = pd.read_csv("../../data/alle_fahrradzaehler_daten.csv")
data.head()

,timestamp,iso_timestamp,zählstand,stand,standort,channel_name,channel_id,counter_site,counter_site_id,domain_name,domain_id,longitude,latitude,timezone,interval,counter_serial
0,2013-01-01T01:00:00+0000,2013-01-01T01:00:00+0100,15,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
1,2013-01-01T02:00:00+0000,2013-01-01T02:00:00+0100,17,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
2,2013-01-01T03:00:00+0000,2013-01-01T03:00:00+0100,14,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
3,2013-01-01T04:00:00+0000,2013-01-01T04:00:00+0100,13,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301
4,2013-01-01T05:00:00+0000,2013-01-01T05:00:00+0100,9,0,Stadt Karlsruhe,Erbprinz. West,101004165,Erbprinzenstraße,100004165,Stadt Karlsruhe,752,8.402715,49.007286,(UTC+01:00) Europe/Paris;DST,15,Y2H16070301


In [7]:
print("Anzahl aller Standorte:", len(get_all_locations(data)))

Anzahl aller Standorte: 111


In [5]:
# channel, die "Auto" oder "PKW" im Namen haben
auto_channels = [col for col in data['channel_name'].unique() if 'Auto' in col or 'auto' in col or 'PKW' in col or 'pkw' in col or 'Car' in col or 'car' in col or 'Vehicle' in col or 'vehicle' in col or 'Fahrzeug' in col or 'fahrzeug' in col]
print("Channels mit 'Auto' oder 'PKW' im Namen:")  
print(auto_channels)

Channels mit 'Auto' oder 'PKW' im Namen:
['Charlottenstraße PKW zur Berufsschule', 'Charlottenstraße PKW zur Planie', 'Metzgerstraße PKW', 'Bellinostraße PKW', 'Hindenburgstraße PKW', 'Bismarckstraße PKW Stadteinwärts', 'Bismarckstraße PKW Stadtauswärts']


In [ ]:
normal_channels = get_normal_channels()

normal_data = filter_data_by_channels(normal_channels)
normal_data.head()

normal_locations = get_all_locations(normal_data)
print("Anzahl der Standorte in den unauffälligen Channels:", len(normal_locations))
normal_locations.head(20)

auto_data = filter_data_by_channels(auto_channels)
auto_locations = get_all_locations(auto_data)
print("Anzahl der Standorte in den Auto-Kanälen:", len(auto_locations))

Anzahl der Standorte in den unauffälligen Channels: 111
Anzahl der Standorte in den Auto-Kanälen: 5


In [11]:
print(auto_locations)
print(auto_channels)

                domain_name  domain_id      counter_site  counter_site_id  \
5767970    Stadt Reutlingen       7581  Charlottenstraße        300017241   
5773010    Stadt Reutlingen       7581     Metzgerstraße        300017243   
5774522    Stadt Reutlingen       7581     Bellinostraße        300017245   
5777546    Stadt Reutlingen       7581  Hindenburgstraße        300017246   
14254155  Stadt Ludwigsburg       6011    Bismarckstraße        300015617   

           latitude  longitude  
5767970   48.494327   9.220319  
5773010   48.490970   9.214630  
5774522   48.486060   9.208770  
5777546   48.487650   9.213265  
14254155  48.899039   9.181309  
['Charlottenstraße PKW zur Berufsschule', 'Charlottenstraße PKW zur Planie', 'Metzgerstraße PKW', 'Bellinostraße PKW', 'Hindenburgstraße PKW', 'Bismarckstraße PKW Stadteinwärts', 'Bismarckstraße PKW Stadtauswärts']
